In [14]:
# 1. Design model (input,output size, forward pass)
# 2. Construct loss and optimizer
# 3. Training loop
#    - forward pass: computer cost
#    - backward pass: compute gradients
#    - update weights
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [15]:
#Load data
bc = datasets.load_breast_cancer()
X,y = bc.data, bc.target
n_samples, n_features = X.shape
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1234)

In [16]:
#Scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train) #fit and transform
X_test = sc.transform(X_test) #just transform

X_train = torch.from_numpy(X_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [17]:
#model
class LogisticRegressionModel(nn.Module):
    
    def __init__(self,n_features):
        super(LogisticRegressionModel,self).__init__()
        self.linear = nn.Linear(n_features,1)
    
    def forward(self,x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = LogisticRegressionModel(n_features)

In [20]:
#loss and optimier
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [22]:
#Traning
num_epochs = 100
for epoch in range(num_epochs):
    #forward and loss
    y_pred = model(X_train)
    loss = criterion(y_pred,y_train)
    #Back
    loss.backward()
    #weight update
    optimizer.step()
    #empty gradients
    optimizer.zero_grad()
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1},t\ loss = {loss.item()}')

epoch: 10,t\ loss = 0.5740951895713806
epoch: 20,t\ loss = 0.4688141345977783
epoch: 30,t\ loss = 0.40386340022087097
epoch: 40,t\ loss = 0.3596094250679016
epoch: 50,t\ loss = 0.3272097408771515
epoch: 60,t\ loss = 0.3022539019584656
epoch: 70,t\ loss = 0.2823050618171692
epoch: 80,t\ loss = 0.2659028172492981
epoch: 90,t\ loss = 0.25211548805236816
epoch: 100,t\ loss = 0.24031883478164673


In [23]:
#Evaluation
with torch.no_grad(): #Similar to detach, defuses gradient storage for the model
    y_pred = model(X_test)
    y_pred_classes = y_pred.round() #round preds to 0 or 1
    acc = y_pred_classes.eq(y_test).sum() / float(y_test.shape[0])
    print(f'Test acc: {acc:.4f}')
    

Test acc: 0.9211
